In [1]:
### This code is for binary classification (water / no water) in water streams

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from PIL import Image
import pandas as pd

## Binary Streamflow Dataset
# labels are converted to 1 (water) and 0 (no water), the last two labels ('poor quality' and 'not working') are discarded
binary_class_map = {0:1, 1:0, 2:0, 3:1}
class BinaryStreamFlowDataset(Dataset):
    def __init__(self, excel_file, transform=None):
        self.data = []
        xls = pd.ExcelFile(excel_file)

        # each sheet = one label
        for idx, sheet_name in enumerate(xls.sheet_names):
            if idx > 3: break
            label = binary_class_map[idx]
            df = pd.read_excel(excel_file, sheet_name=sheet_name)
            for img_path in df['Image_Path'].dropna():
                img_path = img_path.replace('D:', '../images')
                img_path = img_path.replace('\\', '/')
                self.data.append((img_path, label))

        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# dataset = BinaryStreamFlowDataset('../data/image_inventory_cam_1000.xlsx')

In [2]:
# Standard preprocessing for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std=[0.229, 0.224, 0.225]    # ImageNet std
    ),
])

dataset = BinaryStreamFlowDataset("../data/image_inventory_cam_1000.xlsx", transform=transform)

## Split
train_size = int(0.7 * len(dataset))   # 70%
val_size   = int(0.15 * len(dataset))  # 15%
test_size  = len(dataset) - train_size - val_size  # remaining 15%

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # for reproducibility
)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
import torchvision.models as models
import torch.nn as nn

# Load pretrained ResNet-50
embedder = models.resnet50(pretrained=True)
embedder.fc = nn.Identity()    #output embeddings
embedder.eval()

# Small classification head
classifier = nn.Sequential(
    nn.Linear(2048, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.3),

    nn.Linear(256, 32),
    nn.BatchNorm1d(32),
    nn.ReLU(),
    nn.Dropout(0.3),

    nn.Linear(32, 2)
)

/home/jans26/.conda/envs/streamflow-env/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jans26/.conda/envs/streamflow-env/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import wandb
from tqdm import tqdm

# --- Initialize Weights & Biases ---
wandb.init(
    project="water_existence",
    name="test run 9/1",
    config={
        "learning_rate": 1e-3,
        "epochs": 10,
        "batch_size": 32,
        "optimizer": "Adam"
    }
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedder.to(device)
classifier.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=wandb.config.learning_rate)

def get_embeddings(images):
    with torch.no_grad():
        feats = embedder(images)  # [batch, 2048, 1, 1]
        feats = feats.view(feats.size(0), -1)  # flatten -> [batch, 2048]
    return feats

# --- Training Loop ---
for epoch in range(wandb.config.epochs):
    # Training
    classifier.train()
    total_loss, correct, total = 0, 0, 0

    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", leave=False)
    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)

        embeddings = get_embeddings(images)
        outputs = classifier(embeddings)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        train_bar.set_postfix(loss=loss.item())

    train_acc = correct / total
    train_loss = total_loss / total

    # Validation
    classifier.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]", leave=False)
    with torch.no_grad():
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            embeddings = get_embeddings(images)
            outputs = classifier(embeddings)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

            val_bar.set_postfix(loss=loss.item())

    val_acc = val_correct / val_total
    val_loss = val_loss / val_total

    # --- Logging ---
    print(f"Epoch {epoch+1}: "
          f"Train Loss {train_loss:.4f}, Train Acc {train_acc:.4f}, "
          f"Val Loss {val_loss:.4f}, Val Acc {val_acc:.4f}")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

# Saving model (classification head)
torch.save(classifier.state_dict(), 'water_existence_classifier_binary_9_1.pth')

Epoch 1 [Train]:  68%|██████▊   | 216/316 [16:02<07:24,  4.44s/it, loss=0.137] 

In [ ]:
### test loop (standalone)

test_loss, test_correct, test_total = 0, 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        embeddings = get_embeddings(images)
        outputs = classifier(embeddings)
        loss = criterion(outputs, labels)

        test_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

test_acc = test_correct / test_total
test_loss = test_loss / test_total

print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")